In [12]:
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
from __future__ import unicode_literals
import requests
from bs4 import BeautifulSoup as soup
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline


In [13]:
df = pd.read_excel('ilovepdf_merged.xlsx')


In [14]:
df = df.iloc[:,:-2]


In [15]:
col_name = list(df.loc[4,:])

In [16]:
col_name[1]='Area'
col_name[2]='Sex'

In [17]:
df.columns = col_name


In [18]:
df = df[df['Code']!='選區號碼']
df = df[df['Code']!='Code']
df = df[df['Area']!='選區']

In [19]:
df.head()

,Code,Area,Sex,18-20,21-25,26-30,31-35,36-40,41-45,46-50,51-55,56-60,61-65,66-70,71+
2,Constituency,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,Constituency,Sex,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,男,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,NaN,NaN,36,104,108,122,165,188,201,214,224,219,194,386
7,NaN,中環,Male,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [20]:
num = df[(df['18-20'].notnull())&(df['18-20']!='18-20')]


In [21]:
num['Total'] = num.apply(lambda x: x[3:].sum(),axis=1)
num = num[num['Total']<20000]
num.drop('Total',axis=1,inplace=True)


In [22]:
num.index = range(len(num))

In [23]:
num['Sex'] = pd.Series(['M','F']*(int(len(num)/2)))

In [24]:
k = pd.Series(df['Code'].unique())
k = k[k.apply(lambda x: True if type(x)==str else False)]
k = k[k.apply(lambda x: True if len(x)==3 else False)]
n = []
for i in k:
    n.append(i)
    n.append(i)
num['Code'] = pd.Series(n)

In [25]:
k = pd.Series(df['Area'].unique())
k = k[k.notnull()]
k = k[(k!='總計') & (k!='Total') & (k!='Constituency') & (k!='選區') & (k!='Tai Hang Sai')]
k = k.apply(lambda x: x+' Tai Hang Sai' if x=='Nam Shan, Tai Hang Tung &' else x)
k.index = range(len(k))
for i in range(int(len(k)/2)):
    k[i*2]=k[i*2+1]
num['Area'] = k  

In [26]:
district = dict(A = 'Central & Western',
                B = 'Wan Chai',
               C = 'Eastern',
               D = 'Southern',
               E = 'Yau Tsim Mong',
               F = 'Sham Shui Po',
               G = 'Kowloon City',
               H = 'Wong Tai Sin',
               J = 'Kwun Tong',
               K = 'Tsuen Wan',
               L = 'Tuen Mun',
               M = 'Yuen Long',
               N = 'North',
               P = 'Tai Po',
               Q = 'Sai Kung',
               R = 'Sha Tin', 
               S = 'Kwai Tsing',
               T = 'Islands')



In [27]:
num['District'] = num['Code'].apply(lambda x: district[x[0]])

In [28]:
def main_dist(code):
    area_id = code[0]
    if area_id in list('ABCD'):
        out = 'HK Island'
    elif area_id in list('EFG'):
        out = 'Kowloon West'
    elif area_id in list('HJ'):
        out = 'Kowloon East'
    elif area_id in list('KLMST'):
        out = 'New Territories West'
    else:
        out = 'New Territories East'
    return out

num['Main_District'] = num['Code'].apply(main_dist)



In [29]:
num['stat_id']=num.index

In [30]:
num.columns = ['Code','Area', 'Sex', 'age18_20', 'age21_25', 'age26_30', 'age31_35',
           'age36_40','age41_45', 'age46_50', 'age51_55', 'age56_60', 'age61_65', 'age66_70', 'age71_','District','Main_District','stat_id']

In [31]:
num = num[['stat_id','Code','Main_District', 'District', 'Area', 'Sex', 'age18_20', 'age21_25', 'age26_30', 'age31_35',
           'age36_40','age41_45', 'age46_50', 'age51_55', 'age56_60', 'age61_65', 'age66_70', 'age71_']]

In [32]:
num.head()

,stat_id,Code,Main_District,District,Area,Sex,age18_20,age21_25,age26_30,age31_35,age36_40,age41_45,age46_50,age51_55,age56_60,age61_65,age66_70,age71_
0,0,A01,HK Island,Central & Western,Chung Wan,M,36,104,108,122,165,188,201,214,224,219,194,386
1,1,A01,HK Island,Central & Western,Chung Wan,F,40,99,112,153,156,174,195,229,249,233,209,367
2,2,A02,HK Island,Central & Western,Mid Levels East,M,59,178,177,183,269,238,302,301,387,395,281,482
3,3,A02,HK Island,Central & Western,Mid Levels East,F,63,148,183,205,253,280,343,370,439,379,268,525
4,4,A03,HK Island,Central & Western,Castle Road,M,79,183,207,193,252,311,345,392,464,412,325,511


In [33]:
num[num['Code'].apply(lambda x: x[0]=='L')]


,stat_id,Code,Main_District,District,Area,Sex,age18_20,age21_25,age26_30,age31_35,age36_40,age41_45,age46_50,age51_55,age56_60,age61_65,age66_70,age71_
468,468,L01,New Territories West,Tuen Mun,Tuen Mun Town Centre,M,125,310,402,406,526,440,342,399,528,419,279,329
469,469,L01,New Territories West,Tuen Mun,Tuen Mun Town Centre,F,104,280,396,471,503,441,432,478,515,387,218,328
470,470,L02,New Territories West,Tuen Mun,Siu Chi,M,148,407,529,498,541,439,330,400,702,646,550,623
471,471,L02,New Territories West,Tuen Mun,Siu Chi,F,150,417,516,545,556,438,370,526,777,728,455,577
472,472,L03,New Territories West,Tuen Mun,On Ting,M,107,327,398,292,409,471,226,314,420,503,686,911
473,473,L03,New Territories West,Tuen Mun,On Ting,F,107,313,388,261,443,405,320,416,534,835,864,755
474,474,L04,New Territories West,Tuen Mun,Siu Tsui,M,158,362,445,436,694,703,411,571,668,608,665,797
475,475,L04,New Territories West,Tuen Mun,Siu Tsui,F,123,379,401,405,685,685,542,628,747,801,747,688
476,476,L05,New Territories West,Tuen Mun,Yau Oi South,M,123,364,429,250,408,365,233,246,357,388,545,709
477,477,L05,New Territories West,Tuen Mun,Yau Oi South,F,139,360,375,246,328,327,305,363,434,593,670,637


In [34]:
import sqlite3 as sql3


In [35]:
conn = sql3.connect('election.db')


In [36]:
num.to_sql('voters_stat',conn,if_exists='replace',index=False)


In [37]:
c = conn.cursor

In [38]:
url = 'https://www.elections.gov.hk/dc2019/eng/results_hk.html'
html = requests.get(url).text
scrap = soup(html,'html.parser')

In [39]:
col = [i.text for i in scrap.find('table').find_all('th')]

In [40]:
data = []
for row in scrap.find('table').find_all('tr')[1:]:
    el = [i.text for i in row.find_all('td')]
    if len(el)==5:
        app = el
    else:
         app  = data[-1][:2]+el
    data.append(app)

In [41]:
count = 0
for i in data:
    if len(i)!=5:
        count+=1
count

0

In [42]:
vote = pd.DataFrame(data,columns=col)

In [43]:
def vote_to_num(str_num):
    out = ''
    for i in str_num:
        if (i!=',') & (i!='*'):
            out+=i
    out = int(out)
    return out

vote['No. of Votes Received'] = vote['No. of Votes Received'].apply(vote_to_num)

In [44]:
vote['result_id'] = vote.index

In [45]:
vote.columns = ['Code', 'Constituency', 'Candidate_No.',
       'Candidate_Name', 'Votes_No.', 'result_id']
vote = vote[['result_id', 'Code', 'Constituency', 'Candidate_No.',
       'Candidate_Name', 'Votes_No.']]

def find_brac(name):
    key=0
    for i in range(len(name)):
        if name[i]=='(':
            key=i
    if key==0:
        return name
    else:
        return name[:key]

vote['Candidate_Name'] = vote['Candidate_Name'].apply(find_brac)
vote['Candidate_Name'][637] = 'SZE TO POK MAN'
vote['Candidate_Name'][997] = 'HUANG SI HONG'

In [46]:
vote.head()

,result_id,Code,Constituency,Candidate_No.,Candidate_Name,Votes_No.
0,0,A01,CHUNG WAN,1,HUI CHI FUNG,1618
1,1,A01,CHUNG WAN,2,WONG CHUNG WAI,1319
2,2,A02,MID LEVELS EAST,1,MOK KAM SUM,1993
3,3,A02,MID LEVELS EAST,2,NG SIU HONG,2672
4,4,A03,CASTLE ROAD,1,CHENG LAI KING,2669


In [47]:
vote.to_sql('result',conn,if_exists='replace',index=False)

In [48]:
candidate = pd.read_csv('vote4hkpublic-1999-2019.csv')
candidate = candidate[candidate['year']==2019]
def camp_def(string):
    if string=='民主':
        return 'DEM'
    elif string=='建制':
        return 'BEJ'
    else:
        return 'IND'
    
candidate['camp'] = candidate['camp'].apply(camp_def)
candidate['name_en'] = candidate['name_en'].apply(lambda x: x.upper().replace('-',' '))
candidate.drop(4400,inplace=True)
candidate.drop(4401,inplace=True)

In [50]:
candidate.head(3)

,id,year,person_id,name_zh,name_en,election_type,cacode,constituency_id,age,camp,...,is_won,fb_id,ig_id,occupation_zh,occupation_en,political_affiliation_zh,political_affiliation_en,electoral_message_zh,electoral_message_en,email_or_website
4402,4403,2019,1400,梁福元,LEUNG FUK YUEN,by-election,M10,1872,61.0,BEJ,...,True,n/a,NaN,商人,NaN,NaN,NaN,NaN,NaN,NaN
4403,4404,2019,2443,文家駒,MAN KA KOY,by-election,M32,1894,34.0,IND,...,True,mankakoy,NaN,花式跳繩教練,NaN,獨立候選人,NaN,NaN,NaN,NaN
4404,4405,2019,2453,文嘉豪,MAN KA HO DONALD,by-election,M32,1894,42.0,IND,...,False,n/a,NaN,註冊外國律師,NaN,NaN,NaN,NaN,NaN,NaN


In [51]:
candidate.to_sql('candidate',conn,if_exists='replace',index=False)

In [52]:
rate = pd.read_csv('vote_rate.csv',index_col=None)
rate['Electoral Size'] = rate['Electoral Size'].apply(vote_to_num)
rate['Aggregate Voter Turnout'] = rate['Aggregate Voter Turnout'].apply(vote_to_num)
rate['rate_id'] = rate.index


In [53]:
rate.columns =['district', 'electoral_size', 'aggregate_voter_turnout',
       'turnout_rate', 'rate_id']
rate = rate[['rate_id', 'district', 'electoral_size', 'aggregate_voter_turnout',
       'turnout_rate']]

In [54]:
rate.head()

,rate_id,district,electoral_size,aggregate_voter_turnout,turnout_rate
0,0,Central & Western,110483,77277,69.94
1,1,Wan Chai,74839,50846,67.94
2,2,Eastern,323725,233287,72.06
3,3,Southern,151994,107946,71.02
4,4,Yau Tsim Mong,129150,89115,69.00


In [55]:
rate.to_sql('vote_rate',conn,if_exists='replace',index=False)

In [56]:
conn.close()